In [ ]:
#@title pgm1 { form-width: "5%" }
# """
# 1. Implement and demonstrate the FIND-S algorithm for finding the most specific
# hypothesis based on a given set of training data samples. Read the training data from a
# .CSV file.

# Algo:
# 1. Load dataset
# 2. Initiliase h to most specific hyp in H.
# 3. for each positive training instance x:
#         for each attribute constraint ai in x:
#             if the constraint is satisfied by x then do nothing
#             else replace ai in h with next most general constraint that is satisfied by x
# 4. display h
# """

import csv
data = []

with open('dataset1.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    data = list(reader)
    data.pop(0)  # removing headers from dataset
num_attributes = len(data[0])-1 # we don't want last col which is target concet ( yes/no)

print("The training data is as follows")
print(data)
S = ['phi'] * 6
print ("\nThe most specific hypothesis :",S,"\n")
print ("\nThe most general hypothesis : ['?','?','?','?','?','?']\n")

S = data[0][:-1].copy();

for i in range(len(data)):
    if data[i][-1] == 'Yes':
        for j in range(num_attributes):
            if S[j] != data[i][j]:
                S[j] = '?'
    print("\nFor Training example ", i ,"the hypothesis is " , S)
print("\n The Maximally Specific Hypothesis for a given Training Examples :\n\n",S)

In [ ]:
#@title pgm2 { form-width: "5%" }

# Algo
# Step1: Load Data set
# Step2: Initialize General Hypothesis  and Specific  Hypothesis.
# Step3: For each training example
# Step4: If example is positive example
#           if attribute_value == hypothesis_value:
#              Do nothing
#           else:
#              replace attribute value with '?' (Basically generalizing it)
# Step5: If example is Negative example
#           Make generalize hypothesis more specific.

import csv
data = []
print("\n The Given Training Data Set \n")

with open('dataset1.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    data = list(reader)
    data.pop(0)
num_attributes = len(data[0])-1 # we don't want last col which is target concet ( yes/no)

print(data)
S = ['phi'] * num_attributes
temp = ['?'] * num_attributes
print ("\n The most specific hypothesis S0 : ",S,"\n")
print (" \n The most general hypothesis G0 : ",G,"\n")

S = data[0][:-1].copy();
# Comparing with Remaining Training Examples of Given Data Set

print("\n Candidate Elimination algorithm  Hypotheses Version Space Computation\n")
G=[]

for i in range(0,len(data)):
    if data[i][-1]=='Yes':
        for j in range(0,num_attributes):
            if data[i][j]!=S[j]:
                S[j]='?'

        for j in range(0,num_attributes):
            for k  in range(0,len(G)):
                if G[k][j] != '?' and G[k][j] != S[j]:
                    del G[k] #remove it if it's not matching with the specific hypothesis

    else: 
        for j in range(0,num_attributes):
             if S[j] != data[i][j] and S[j]!= '?':  #if not  matching with the specific Hypothesis take it seperately and store it 
                 temp[j]=S[j]
                 G.append(temp) # this is the version space to store all Hypotheses
                 temp = ['?'] * num_attributes
    
    print("S",i+1,": ",S)
    
    if (len(G)==0):
        print("G",i+1,": ",temp)
    else:
         print("G",i+1,": ",G)

In [ ]:
#@title pgm3 { form-width: "5%" }
import csv
import pandas as pd
from math import log2
import operator
from pandas.core.frame import DataFrame


def loadCsv(filename):
    with open(filename) as f:
        data = csv.reader(f)
        lines = list(data)
    return lines


data = loadCsv("dataset.csv")
attrs = data[0]
vals = data[1:]
d = pd.DataFrame(vals, columns=attrs)


def entropy(P, N):
    if P == 0 or N == 0:
        return 0
    # Formula for Entropy:
    entr_p = ((-P) / (P + N)) * log2((P) / (P + N))
    entr_n = ((-N) / (P + N)) * log2((N) / (P + N))
    entr = entr_p + entr_n
    return entr


def entropy_dataset(data):
    f = data["PlayTennis"]
    P = sum(int(x == "Yes") for x in f)
    N = len(f) - P
    return (entropy(P, N), P, N)


def entropy_attr(data: DataFrame, P, N):

    cols = data.columns[:-1]
    entropy_dict = {K: None for K in cols}

    for col in cols:
        coldata = data[[col, "PlayTennis"]]
        dset = {K: [0, 0] for K in set(coldata[col])}
        eset = {K: None for K in set(coldata[col])}

        # Count Sub-Attr Against Yes or No from PlayTennis
        for (value, outcome) in coldata.itertuples(index=False, name=None):
            if outcome == "Yes":
                dset[value][0] += 1
            else:
                dset[value][1] += 1

        # Calculate Entropy for Each, Formula.
        for (subattr, calc) in dset.items():
            p = (calc[0] + calc[1]) / (P + N)
            p = p * entropy(calc[0], calc[1])
            eset[subattr] = p

        # Finally, Push the Sum into the Main Array.
        entropy_dict[col] = sum(eset.values())

    return entropy_dict

# Gain Formula
def gain(data_entropy, attr_entropy):
    gain = {K: data_entropy - attr_entropy[K] for K in attr_entropy.keys()}
    return gain


def create_tree(data, name, indent):

    # Check if Each is just Yes or No
    if all(x == "Yes" for x in data["PlayTennis"]):
        print("{} Yes".format(indent, name))
        return

    if all(x == "No" for x in data["PlayTennis"]):
        print("{} No".format(indent, name))
        return

    # Calculate for Each step of the Way.
    entrd, P, N = entropy_dataset(data)
    avg_info_entropy = entropy_attr(data, P, N)
    gain_data = gain(entrd, avg_info_entropy)

    # Just get the Max value from gain_data.
    max_val = max(gain_data.items(), key=operator.itemgetter(1))[0]

    print(indent + max_val)
    indent += "  "

    # Do this groupby
    grouped = data.groupby(max_val)

    for group in grouped.groups.keys():
        print(indent + group)
        new_table = grouped.get_group(group)
        create_tree(new_table, group, indent + "  ")


create_tree(d, "root", "")

# """
# Outlook
#   Overcast
#      Yes
#   Rainy
#     Windy
#       False
#          Yes
#       True
#          No
#   Sunny
#     Humidity
#       High
#          No
#       Normal
#          Yes
# """

In [ ]:
#@title pgm4 { form-width: "5%" }
import numpy as np 
X = np.array(([2, 9], [1, 5], [3, 6]), dtype=float) # Features ( Hrs Slept, Hrs Studied)
y = np.array(([92], [86], [89]), dtype=float) # Labels(Marks obtained)
c=np.amax(X,axis=0) # Normalize
print(c)
X = X/c # Normalize
y = y/100
print(X)
print(y)

def sigmoid(x):
    return 1/(1 + np.exp(-x))
def sigmoid_grad(x):
    return x * (1 - x)

 # Variable initialization
epoch=1000 #Setting training iterations
eta =0.1 #Setting learning rate (eta)
input_neurons = 2 #number of features in data set
hidden_neurons = 3 #number of hidden layers neurons
output_neurons = 1 #number of neurons at output layer
# Weight and bias - Random initialization
wh=np.random.uniform(size=(input_neurons,hidden_neurons)) # 2x3
print(wh)
bh=np.random.uniform(size=(1,hidden_neurons)) # 1x3
print(bh)
wout=np.random.uniform(size=(hidden_neurons,output_neurons)) # 3x1
print(wout)
bout=np.random.uniform(size=(1,output_neurons))
print(bout)

for i in range(epoch):
    #Forward Propogation
    h_ip=np.dot(X,wh) + bh # Dot product + bias
    print(h_ip)
    h_act = sigmoid(h_ip) # Activation function
    o_ip=np.dot(h_act,wout) + bout
    output = sigmoid(o_ip)
    
    #Backpropagation
    # Error at Output layer
    Eo = y-output # Error at o/p
    outgrad = sigmoid_grad(output)
    d_output = Eo* outgrad # Errj=Oj(1-Oj)(Tj-Oj)
    print("the d_output is",d_output)

    # Error at Hidden later
    Eh = d_output.dot(wout.T) # .T means transpose
    hiddengrad = sigmoid_grad(h_act) # How much hidden layer wts contributed to error
    d_hidden = Eh * hiddengrad
    wout += h_act.T.dot(d_output) *eta # Dotproduct of nextlayererror and currentlayerop
    wh += X.T.dot(d_hidden) *eta

print("Normalized Input: \n" + str(X))
print("Actual Output: \n" + str(y))
print("Predicted Output: \n" ,output)

In [69]:
#@title pgm5 { form-width: "5%" }

# for gausian baye dataset only, skipped binomial baye
import math
import pandas as pd
from functools import partial

#filename = 'pima-indians-diabetes.csv'
data = pd.read_csv("pima-indians-diabetes.csv")
data.columns = [
    "Pregnancies",
    "Glucose",
    "BloodPressure",
    "SkinThickness",
    "Insulin",
    "BMI",
    "DiabetesPedigreeFunction",
    "Age",
    "Class",
]

# Gauissian Bayes.
# Class is the Deciding Factor
def GaussianFunction(mean, std, x):
    return (1 / (math.sqrt(2 * math.pi) * std)) * math.pow(math.e, -math.pow(x - mean, 2) / (2 * std ** 2))


def freq_table(data: pd.DataFrame):
    vnf = {}
    for col in data.columns[:-1]:
        values = data[[col, "Class"]]
        P = values.loc[values["Class"] == 1]
        N = values.loc[values["Class"] == 0]
        vnf[col] = [
            partial(GaussianFunction, P[col].mean(), P[col].std()),
            partial(GaussianFunction, N[col].mean(), N[col].std()),
        ]
    return vnf


def pred(data: pd.DataFrame, fq):
    predictions = []
    for row in data.to_numpy():
        Pprob = 1
        Nprob = 1
        for (col, val) in zip(data.columns, row):
            Pprob = Pprob * fq[col][0](val)
            Nprob = Nprob * fq[col][1](val)
        predictions.append(int(Pprob > Nprob))
    return predictions


def get_Accuracy(actual, predicted):
    count_true = 0
    total = len(predicted)
    for x, y in zip(actual, predicted):
        count_true += (x == y)
    return count_true / total


ratio = 0.5
intratio = int(ratio * len(data))
train, test = data[:intratio], data[intratio:]
vtable = freq_table(train)
prediction = pred(test.drop(columns="Class"), vtable)
acc = get_Accuracy(test.iloc[:, -1], prediction)
print("Acc is ",acc * 100)

Acc is  76.30208333333334


In [ ]:
#@title pgm6 { form-width: "5%" }
from sklearn.datasets import fetch_20newsgroups
import sklearn.datasets as skd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



# Get Some categories of Study
categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]

# Get Train Dataset for categories
twenty_train = fetch_20newsgroups(subset="train", categories=categories, shuffle=True)
# Get the Test Dataset for the same.
twenty_test = fetch_20newsgroups(subset="test", categories=categories, shuffle=True)

# Add in the Local Conditions to, Local Downloads.
# news_train=skd.load_files(r"/home/student/Desktop/ML/20news-bydate-train",categories=categories,encoding='ISO-8859-1')
# news_test=skd.load_files(r"/home/student/Desktop/ML/20news-bydate-test",categories=categories,encoding='ISO-8859-1')

# Vectorizers
cvec = CountVectorizer()
tvec = TfidfTransformer()

# Vectorize the Data.
X_Train = cvec.fit_transform(twenty_train.data)
X_Train_TFIDF = tvec.fit_transform(X_Train)

# Vectorize the Tests
X_Test = cvec.transform(twenty_test.data)
X_Test_TFIDF = tvec.transform(X_Test)


# Make Model
Model = MultinomialNB()

# Train Model
Model.fit(X_Train_TFIDF, twenty_train.target)

# Make Predictions

X_Pred = Model.predict(X_Test_TFIDF)

print("Accuracy:", accuracy_score(twenty_test.target, X_Pred))
print("Confusion matrix is \n", confusion_matrix(twenty_test.target, X_Pred))
print(classification_report(twenty_test.target, X_Pred, target_names=twenty_test.target_names))

# prediction
doc=['god is love','openGL on the GPU is fast']
X_new_counts=cvec.transform(doc)
X_new_tfid=tvec.transform(X_new_counts)
predicted=Model.predict(X_new_tfid)
for x in predicted:
    print(x)

In [ ]:
#@title pgm7 { form-width: "5%" }
! pip install pgmpy
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
heartDisease = pd.read_csv('p.csv', names=['preg', 'glucose', 'bp', 'skinthick', 'insulin', 'bmi', 'diapedigree', 'age', 'class'])
print(heartDisease.head())
model = BayesianModel([('preg', 'glucose'), ('bp', 'skinthick'),('insulin', 'bmi'), ('bmi', 'diapedigree'), ('age', 'class'), ('insulin', 'class')])
model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)
print(model.get_cpds('preg'))
heart_infer = VariableElimination(model)
print("Probability of heart disease given age = 28\n")
q = heart_infer.query(variables=['class'], evidence={'age': 28})
print(q)
print("Probability of heart disease given bp = 60\n")
q = heart_infer.query(variables=['class'], evidence={'insulin': 60})
print(q)

In [ ]:
#@title pgm8 { form-width: "5%" }
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score,confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

# import some data to play with
iris = datasets.load_iris()
print("\n IRIS FEATURES :\n",iris.feature_names)
print("\n IRIS TARGET :\n",iris.target)
print("\n IRIS TARGET NAMES:\n",iris.target_names)

# Store the input
# This is what KMeans thought model.labels_ as a Pandas Dataframe and set the column names
X = pd.DataFrame(iris.data)
X.columns = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width']
y = pd.DataFrame(iris.target)
y.columns = ['Targets']

# Set the size of the plot and Create a colormap
plt.figure(figsize=(14,7))
colormap = np.array(['red', 'lime', 'black'])
 
# Plot Orginal
plt.subplot(1, 2, 1)
plt.scatter(X.Petal_Length, X.Petal_Width, c=colormap[y.Targets], s=40)
plt.title('Real Classification')

# Build model
# K Means Cluster
km = KMeans(n_clusters=3)
predY_km = km.fit_predict(X)
# Rearrange based on what output you get from realistic classification
predY_km = np.choose(predY_km, [2, 0, 1]).astype(np.int64)
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.scatter(X.Petal_Length,X.Petal_Width, c=colormap[predY_km], s=40)
plt.title('K Mean Classification')

# Performance Metrics
print(accuracy_score(y, predY_km))
# Confusion Matrix
print(confusion_matrix(y, predY_km))


# gausianmodel
gmm = GaussianMixture(n_components=3)
predY_gmm = gmm.fit_predict(X)

# Rearrange based on what output you get from realistic classification
predY_gmm = np.choose(predY_gmm, [1, 0, 2]).astype(np.int64)
plt.figure(figsize=(14,7))
plt.subplot(1, 2, 1)
plt.scatter(X.Petal_Length, X.Petal_Width, c=colormap[predY_gmm], s=40)
plt.title('GMM Classification')

print(accuracy_score(y, predY_gmm))
print(confusion_matrix(y, predY_gmm))

In [ ]:
#@title pgm9 { form-width: "5%" }
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix

import pandas as pd
import numpy as np


#load iris dataset
from sklearn.datasets import load_iris
iris=load_iris()
#print(iris)

iris=datasets.load_iris()
iris_data=iris.data
iris_labels=iris.target

x_train, x_test, y_train, y_test= train_test_split(iris_data, iris_labels, test_size=0.20)
classifier=KNeighborsClassifier(n_neighbors=6)
classifier.fit(x_train, y_train)
y_pred=classifier.predict(x_test)

print("condusion matrix:\n",confusion_matrix(y_test, y_pred))
print("\nreport:\n",classification_report(y_test, y_pred))

# Print which one was correct, which one was wrong
d = {"Y Test": y_test, "Y Predicted": y_pred, "Is Correct?":[]}

for x,y in zip(y_test, y_pred):
    d["Is Correct?"].append("Yes" if x==y else "No")

final = pd.DataFrame(d)

print(final)
#
# The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

# The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

# The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

In [ ]:
#@title pgm10 { form-width: "5%" }
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import math

tou = 0.8

X_train =  np.array(list(range(3,33))+[3.2,4.2])[:,None] #np.newaxis # convert 1d to 2d
y_train=np.array([1,2,1,2,1,1,3,4,5,4,5,6,5,6,7,8,9,10,11,11,12,11,11,12,13,16,17,19,18,34,21,22])

X_test = np.array([i/10. for i in range(340)])[:,None]   #np.newaxis # convert 1d to 2d
y_test = []

for r in X_test:
    wt = np.exp(-np.sum((X_train-r)**2,axis=1)/(2*tou)**2)
    w = np.diag(wt)
    teth1 = np.linalg.inv(X_train.T.dot(w).dot(X_train))
    teth2 = X_train.T.dot(w).dot(y_train)
    prediction = r.dot(teth1).dot(teth2)
    y_test.append(prediction)

plt.plot(X_train.squeeze(), y_train, 'o')
plt.plot(X_test.squeeze(), np.array(y_test), 'r-')
#print(y_test)

In [ ]:
#@title mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')